In [8]:
import pandas as pd
import numpy as np
import time
import os
import json
from multiprocessing import Process, Manager, Value, Array, managers
from numpy import genfromtxt
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from multiprocessing import Pool
import collections
import itertools
from itertools import combinations
# from more_itertools import unique_everseen
from Bio.Seq import Seq
from Bio import pairwise2

#  core genome with gap and no alignment

df = pd.read_csv('/data/docker_qiime2_share_F/all_ec_sec_round/1135_cated.csv',
                header = None,
                index_col = 0,
                dtype = {0: 'str',
                         1: 'str'}
                )

In [16]:
with open('strains_id.json', 'r') as f_read:
    read = json.load(f_read)
print(len(read))

inv_map = {str(v): k for k, v in read.items()}

1140


In [ ]:
eq_fasta_path = 'eq_genes/'
final_core_genome_path = "/data/docker_qiime2_share_G/Hejunwei/core_genome_study/CNN/the_core_genome_1127_no_align.fasta"

to_write = []

seq_coll = {}

ini_seq = SeqIO.to_dict(SeqIO.parse(eq_fasta_path + "10001-14085.fasta", 
                                    "fasta")
                       )
for x in ini_seq:
    seq_coll[inv_map[x[0: 5]]] = "-" * 200

for x in os.listdir(eq_fasta_path):
#     print(x)
    x_fasta = SeqIO.to_dict(SeqIO.parse(eq_fasta_path + x, 
                                        "fasta")
                           )
    for y in x_fasta:
        seq_coll[inv_map[y[0: 5]
                        ]
                ] = seq_coll[inv_map[y[0: 5]
                                    ]
                            ] + str(x_fasta[y].seq) + '-' * 200

for z in seq_coll:
    to_write.append(SeqRecord(id = z, 
                              name = z,
                              description = "",
                              dbxrefs=[],
                              seq = Seq(seq_coll[z])
                             )
                   )
with open(final_core_genome_path, "w") as output_handle:
        SeqIO.write(to_write, output_handle, "fasta")

# make core genome with some gene missing for marker finder

## find most different clusters


### algorithm explaination:
varible 1: uniqe seq count or count of 100% clustered representative <br>
v2: amout of each unique seq count<br>
v3 distance between each unique type<br>

eg: sample 1:<br>
seq A count 1126, seq B 1, distance between AB:d_ab<br>
contribution to classification: (B/A)*d_ab<br>

eg: sample 2:<br>
seq A count 400, seq B 200, seq C 200 seq D 100 seq F 217, distance between AB:d_ab<br>
contribution to classification: (B/A)*d_ab*((A+B)/Total_len) + (C/A)*d_ac*((A+C)/Total_len) ....<br>
for all combinations

In [13]:
kaligned_path = 'kaligned/'

def get_distance(str1, str2):
    d = 0
    l = len(str1)
    
    for c in range(len(str1)):
        if str1[c] != str2[c]:
            d = d + 1
    d = d
    
    return d

def get_contribution(in_fasta_path, rt_dict):
    in_fasta = SeqIO.to_dict(SeqIO.parse(kaligned_path + in_fasta_path,
                                         'fasta')
                            )
    
    rep_d = {}
    
    for keys in in_fasta.keys():
        
        seq = str(in_fasta[keys].seq)
        
        if seq in rep_d.keys():
            rep_d[seq].append(keys)
        else:
            rep_d[seq] = [keys]
            
    key_lst = list(rep_d.keys())
    
    calculation_lst = combinations(key_lst, 2)
    
    total_distance = 0
    
    for pairs in calculation_lst:
        a = len(pairs[0])
        b = len(pairs[1])
        if a < b:
            bi_zhi = a/b
        else:
            bi_zhi = b/a
        
        total_distance = bi_zhi*get_distance(pairs[0], pairs[1])*(a + b)*(1/1127)
    
    rt_dict[in_fasta_path.replace('.fasta', '')] = total_distance
    
def divide_chunks(l, n):

    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n] 

In [14]:
start_time_external = time.time()
print(time.asctime())

if __name__ == '__main__':
    procc_num = 16
    
    mission_lst = list(divide_chunks(os.listdir(kaligned_path),
                                     procc_num
                                    )
                      )
    
    manager = Manager()
    return_dict = manager.dict()
    
    for submissions in mission_lst:
    
        jobs = []

        for x in range(0, len(submissions)):
            p = Process(target = get_contribution,
                        args = (submissions[x], 
                                return_dict
                               )
                       )
            p.start()
            jobs.append(p)


        for z in jobs:
            z.join()
        
end_time_external = time.time()
print("final time usage " + str(end_time_external - start_time_external))
print(time.asctime())

Sun Jan 29 12:58:28 2023
final time usage 119.26887249946594
Sun Jan 29 13:00:27 2023


In [15]:
df = pd.DataFrame.from_dict(return_dict, orient='index')
df = df.sort_values(by = [0], ascending = False)
df

,0
10741-13098,3195.677019
11010-10058,2668.720497
10092-12021,2178.037267
10052-13314,2157.790594
10238-10033,2010.489796
...,...
10871-11268,0.337178
11127-10138,0.319432
10235-15114,0.308784
10425-12768,0.273292


In [17]:
df.to_csv('cluster_contribution.csv',
          header = False)

### check if algorithm worked

In [18]:
in_fasta = SeqIO.to_dict(SeqIO.parse(kaligned_path + '10155-13700' + '.fasta',
                                         'fasta')
                            )

In [19]:
rep_d = {}

for keys in in_fasta.keys():
        
    seq = str(in_fasta[keys].seq)

    if seq in rep_d.keys():
        rep_d[seq].append(keys)
    else:
        rep_d[seq] = [keys]
            
key_lst = list(rep_d.keys())

In [20]:
len(key_lst)

162

In [21]:
get_distance(key_lst[0], key_lst[1])

20

## make KO core genome

In [22]:
df = pd.read_csv('cluster_contribution.csv', 
                 header = None)
df

,0,1
0,10741-13098,3195.677019
1,11010-10058,2668.720497
2,10092-12021,2178.037267
3,10052-13314,2157.790594
4,10238-10033,2010.489796
...,...,...
707,10871-11268,0.337178
708,11127-10138,0.319432
709,10235-15114,0.308784
710,10425-12768,0.273292


In [24]:
with open('strains_id.json', 'r') as f_read:
    read = json.load(f_read)
print(len(read))

inv_map = {str(v): k for k, v in read.items()}

1140


In [25]:
a = list(df[0])

a.remove(a[500])

In [27]:
len(a)

712

In [28]:
# to_skip = list(df[0])[2: ]
# to_skip = list(df[0])
# to_skip = a
to_skip = []
len(to_skip)

0

In [29]:
def concat(seq_rec_lst, shared_dict_, trans_d):
    for a_rec in seq_rec_lst:
        the_seq = str(shared_dict_[trans_d[a_rec.id[0: 5]]].seq)
        
        the_seq = the_seq + str(a_rec.seq)
        
        shared_dict_[trans_d[a_rec.id[0: 5]]] = SeqRecord(id = trans_d[a_rec.id[0: 5]], 
                                                             name = trans_d[a_rec.id[0: 5]],
                                                             description = "",
                                                             dbxrefs=[],
                                                             seq = Seq(the_seq)
                                                            )
#         print(shared_dict_[trans_d[a_rec.id[0: 5]]].seq)

def mission_spliter(a_lst, proc_num):
    n = len(a_lst)
    m = n // proc_num
    r = n % proc_num
    sublists = []
    start = 0
    for i in range(proc_num):
        end = start + m
        if i < r:
            end += 1
        sublists.append(a_lst[start:end])
        start = end
    return sublists

In [33]:
print(time.asctime())

proc_num = 16

if __name__ == "__main__":
    ini_seq = SeqIO.to_dict(SeqIO.parse("kaligned/10001-14085.fasta", 
                                    "fasta")
                       )
    
    manager = Manager()
    shared_dict = manager.dict()
    d = inv_map
    
    for x in ini_seq.keys():
        shared_dict[d[x[0: 5]]] = SeqRecord(id = d[x[0: 5]], 
                                         name = d[x[0: 5]],
                                         description = "",
                                         dbxrefs=[],
                                         seq = Seq("")
                                        )
    
    
    for cls in os.listdir('kaligned/'):
        a_cls = SeqIO.to_dict(SeqIO.parse('kaligned/' + cls, 
                                    "fasta")
                       )
        
        m_lst = list(a_cls.values())
        
        splitted_m_lst = mission_spliter(m_lst, proc_num)
        
        jobs = []

        for ms in splitted_m_lst:
            p = Process(target = concat,
                       args = (list(ms), shared_dict, d)
                       )
          
            jobs.append(p)

            p.start()

    #     p.join()

        for z in jobs:
            z.join()
            
    to_save = list(shared_dict.values())
    
    with open("the_core_genome_1127_full.fasta", "w") as output_handle:
        SeqIO.write(to_save, output_handle, "fasta")
        
    
print(time.asctime())

Sun Jan 29 13:09:42 2023


Process Process-8087:
Process Process-8086:
Traceback (most recent call last):
Exception ignored in: <function _releaseLock at 0x7f27b94ad0e0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Process Process-8084:
Process Process-8085:
Process Process-8088:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 290, in _bootstrap
    util._run_after_forkers()
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 811, in _callmethod
    conn = self._tls.connection
  File "/opt/conda/lib/python3.7/multiprocessing/util.py", line 167, in _run_after_forkers
    func(obj)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 890, in _after_fork
    self._incref()
AttributeError: 'ForkAwareLocal' object has n

  File "<string>", line 2, in __getitem__
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 811, in _callmethod
    conn = self._tls.connection
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 802, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 815, in _callmethod
    self._connect()
  File "/tmp/ipykernel_1267/2916765815.py", line 3, in concat
    the_seq = str(shared_dict_[trans_d[a_rec.id[0: 5]]].seq)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 492, in Client
    c = SocketClient(address)
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'
  File "<string>", line 2, in __getitem__
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 802, in _connect
    conn = self._C

FileNotFoundError: [Errno 2] No such file or directory
KeyboardInterrupt
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 802, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 492, in Client
    c = SocketClient(address)
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 811, in _callmethod
    conn = self._tls.connection
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 620, in SocketClient
    s.connect(address)
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'
  File "/tmp/ipykernel_1267/2916765815.py", line 3, in concat
    the_seq = str(shared_dict_[tr

  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
FileNotFoundError: [Errno 2] No such file or directory
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 802, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 811, in _callmethod
    conn = self._tls.connection
Traceback (most recent call last):
  File "/tmp/ipykernel_1267/2916765815.py", line 3, in concat
    the_seq = str(shared_dict_[trans_d[a_rec.id[0: 5]]].seq)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 492, in Client
    c = SocketClient(address)
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'
  File "<string>", line 2, in __getitem__
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py

...

  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 802, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 492, in Client
    c = SocketClient(address)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 620, in SocketClient
    s.connect(address)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
FileNotFoundError: [Errno 2] No such file or directory
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 811, in _callmethod
    conn = self._tls.connection
  File "/tmp/ipykernel_1267/2916765815.py", line 3, in concat
    the_seq = str(shared_dict_[trans_d[a_rec.id[0: 5]]].seq)
  File "<string>", line 2

  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 811, in _callmethod
    conn = self._tls.connection
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 802, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
Traceback (most recent call last):
  File "/tmp/ipykernel_1267/2916765815.py", line 3, in concat
    the_seq = str(shared_dict_[trans_d[a_rec.id[0: 5]]].seq)
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 492, in Client
    c = SocketClient(address)
  File "<string>", line 2, in __getitem__
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()

During handling of the above exception, another exception occurred:

  File "/opt/conda/lib/python3.7/multiprocessin

FileNotFoundError: [Errno 2] No such file or directory
  File "<string>", line 2, in __getitem__

During handling of the above exception, another exception occurred:

Process Process-8269:
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 815, in _callmethod
    self._connect()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 802, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 811, in _callmethod
    conn = self._tls.connection
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 492, in Client
    c = SocketClient(address)
  File "/opt/conda/l

  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 815, in _callmethod
    self._connect()
Traceback (most recent call last):
  File "/tmp/ipykernel_1267/2916765815.py", line 3, in concat
    the_seq = str(shared_dict_[trans_d[a_rec.id[0: 5]]].seq)

During handling of the above exception, another exception occurred:

  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 802, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 811, in _callmethod
    conn = self._tls.connection
Process Process-8277:
  File "<string>", line 2, in __getitem__
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 492, in Client
    c = SocketClient(address)
Traceback (most re


During handling of the above exception, another exception occurred:

Process Process-8282:
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 815, in _callmethod
    self._connect()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 492, in Client
    c = SocketClient(address)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/managers.py", line 802, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 620, in SocketClient
    s.connect(address)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 492, in Client
    c = SocketClient(address)
FileNotFoundError: [Errno 2] No such file or directory
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run

In [34]:
kaligned_path = 'kaligned/'
final_core_genome_path = "the_core_genome_1127_full.fasta"

to_write = []

seq_coll = {}

ini_seq = SeqIO.to_dict(SeqIO.parse(kaligned_path + "10001-14085.fasta", 
                                    "fasta")
                       )
for x in ini_seq:
    seq_coll[inv_map[x[0: 5]]] = ""

for x in os.listdir(kaligned_path):
#     print(x)
    x_fasta = SeqIO.to_dict(SeqIO.parse(kaligned_path + x, 
                                        "fasta")
                           )
    
    if x.replace('.fasta', '') in to_skip:
        for y in x_fasta:
            seq_coll[inv_map[y[0: 5]
                            ]
                    ] = seq_coll[inv_map[y[0: 5]
                                        ]
                                ] + '-' * len(x_fasta[y].seq
                                             )
    else:
        for y in x_fasta:
            seq_coll[inv_map[y[0: 5]
                            ]
                    ] = seq_coll[inv_map[y[0: 5]
                                        ]
                                ] + str(x_fasta[y].seq)

for z in seq_coll:
    to_write.append(SeqRecord(id = z, 
                              name = z,
                              description = "",
                              dbxrefs=[],
                              seq = Seq(seq_coll[z])
                             )
                   )
with open(final_core_genome_path, "w") as output_handle:
        SeqIO.write(to_write, output_handle, "fasta")